<a href="https://colab.research.google.com/github/atw617/Stable-Diffusion/blob/main/my_stable_diffusion_with_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drive Mount (로라나 모델이 드라이브에 저장되어 있을때만)

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive/')

Mounted at /content/drive/


# Requirements (install / import)

In [ ]:
# !pip install xformers
# CUDA version 문제로 xformers와 사용하지 않음
!pip install accelerate
!pip install -q diffusers transformers
!pip install gradio
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.5 M

In [ ]:
import gradio as gr
import torch
import os
import random
from huggingface_hub import HfApi
from pathlib import Path
from PIL import Image
import numpy as np
from diffusers.utils import load_image
from controlnet_aux.lineart import LineartDetector
from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.

# Model Functionize

In [ ]:
# https://huggingface.co/digiplay/RunDiffusionFXPhotorealistic_v1   https://huggingface.co/SG161222/Realistic_Vision_V2.0   https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt
checkpoint_list = ["digiplay/RunDiffusionFXPhotorealistic_v1", "SG161222/Realistic_Vision_V2.0", "runwayml/stable-diffusion-v1-5"]

# https://huggingface.co/ControlNet-1-1-preview/control_v11p_sd15_lineart   https://huggingface.co/lllyasviel
controlnet_list = ["ControlNet-1-1-preview/control_v11p_sd15_lineart", "lllyasviel/control_v11p_sd15_normalbae", "lllyasviel/control_v11p_sd15_seg"]

# https://civitai.com/models/219536/cityscape-and-character-cumulonimbus-cloud   https://civitai.com/models/44667/syfyeye1
lora_list = ["None","cloud_v1.safetensors", "SyFyEye1_v2.0.safetensors"]


def image_gen(checkpoint_selected, controlnet_selected, lora_seleted, image, prompt, negative_prompt, guidance_scale, num_inference_steps):
    image = image.resize((512, 512))
    processor = LineartDetector.from_pretrained("lllyasviel/Annotators")

    control_image = processor(image)

    controlnet = ControlNetModel.from_pretrained(controlnet_selected, torch_dtype=torch.float16)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(checkpoint_selected,controlnet=controlnet, torch_dtype=torch.float16)
    if lora_seleted != "None":
          pipe.load_lora_weights("/content/drive/MyDrive/lora", weight_name = lora_seleted )

    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.enable_model_cpu_offload()

    seed_num = random.randint(0, 10000000)
    generator = torch.manual_seed(seed_num)
    image = pipe(prompt = prompt, negative_prompt = negative_prompt, guidance_scale = guidance_scale, num_inference_steps=num_inference_steps, generator=generator, image=control_image).images[0]

    return image, prompt, negative_prompt

# GUI (Grdio Interface)

In [ ]:
# 인터페이스 생성
# 스테이블 디퓨전 파이프라인 공식 문서 https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/overview 참고
# Gradio 공식 문서 https://www.gradio.app/docs/interface 참고

demo = gr.Interface(
    image_gen,
    inputs = [gr.Dropdown(checkpoint_list, label="Checkpoint model", value='digiplay/RunDiffusionFXPhotorealistic_v1', info="choose checkpoint model"),
              gr.Dropdown(controlnet_list, label="Controlnet",  value='ControlNet-1-1-preview/control_v11p_sd15_linear', info="choose controlnet"),
              gr.Dropdown(lora_list, label="LoRA", value="None", info="choose Lora"),
              gr.Image(type="pil"),
              gr.Textbox(label="prompt"),
              gr.Textbox(label="negative prompt"),
              gr.Slider(minimum = 1, maximum = 30, step = 1, value=7, label="CFG", info="min 1 / max 30"),
              gr.Slider(minimum = 10, maximum = 100, step = 1, value=50, label="num_inference_steps", info="min 10 / max 100")],
    outputs = ["image", gr.Markdown("prompt", show_label = True , label = "prompt"),
                gr.Markdown("negative_prompt", show_label = True, label = "negative_prompt")],
)

# share=True 시 Public Link 로 배포
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

# prompt examples
# (Realistic:1.5) SyFyEye1, dark, standing, lake, lens flare, night sky, standing, winter, waves, shadow, galaxy, planet
# cloud, blue sky, masterpiece, best quality, hyper realistic, realistic, ultra high res, highres , dslr, photorealistic, high detailed, 4k, 8k
# landscape, masterpiece, best quality, hyper realistic, realistic, ultra high res, highres , dslr, photorealistic, high detailed

# negative prompt examples
#  (worst quality, low quality:1.4), (sketch by bad-artist, photography by bad-artist, illustration by bad-artist), blurry, signature, watermark, monochrome, cartoon, painting, lowres, text, bad art, (low quality:1.4), (worst quality:1.4)

/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:163: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: ControlNet-1-1-preview/control_v11p_sd15_linear or set allow_custom_value=True.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b5d4f0975d91573c84.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


sk_model.pth:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

sk_model2.pth:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/diffusers/loaders/lora.py:708: FutureWarning: `_modify_text_encoder` is deprecated and will be removed in version 0.25. You are using an old version of LoRA backend. This will be deprecated in the next releases in favor of PEFT make sure to install the latest PEFT and transformers packages in the future.
  deprecate("_modify_text_encoder", "0.25", LORA_DEPRECATION_MESSAGE)
/usr/local/lib/python3.10/dist-packages/diffusers/loaders/lora.py:679: FutureWarning: `_remove_text_encoder_monkey_patch_classmethod` is deprecated and will be removed in version 0.25. You are using an old version of LoRA backend. This will be deprecated in the next releases in favor of PEFT make sure to install the late

  0%|          | 0/50 [00:00<?, ?it/s]